### 1) Encontrar los datos

In [17]:
import pandas as pd
from functools import reduce
url_1 = 'https://raw.githubusercontent.com/leoferres/gitiele356b/main/data/pobXedadCenso.csv'
pob_censo_edad = pd.read_csv(url_1)
url_2 = 'https://raw.githubusercontent.com/leoferres/gitiele356b/main/data/hdi.csv'
hdi = pd.read_csv(url_2)


urls=[
'vacunacion_comuna_edad_1eraDosis_std.csv',
'vacunacion_comuna_edad_2daDosis_std.csv',
'vacunacion_comuna_edad_UnicaDosis_std.csv']
vacunacion=[]
for i in range(len(urls)):
    vacunacion.append(pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto81/'+ urls[i]))





### 2) Pre-procesamiento y limpieza de los datos:

In [21]:
df_vacunacion = reduce(lambda  left,right: pd.merge(left,right,on=['Region','Codigo region' ,'Comuna','Codigo comuna', 'Edad'],
                                            how='left'), vacunacion)

df_vacunacion["Codigo comuna"] = pd.to_numeric(df_vacunacion["Codigo comuna"])
Edad=df_vacunacion['Edad']
RangoEtareo=[]
eiqueta=['EDAD_0A5','EDAD_6A14','EDAD_15A64','EDAD_65YMAS']
for i in range(df_vacunacion.shape[0]):
    if Edad[i]<=5:
        RangoEtareo.append(eiqueta[0])
    elif (Edad[i]>=6)&(Edad[i]<=14):
        RangoEtareo.append(eiqueta[1])
    elif (Edad[i]>=15)&(Edad[i]<=64):
        RangoEtareo.append(eiqueta[2])
    elif (Edad[i]>=65):
        RangoEtareo.append(eiqueta[3])
        
df_vacunacion['RangoEtareo'] =RangoEtareo
df_vacunacion=df_vacunacion.rename(columns={'Codigo comuna': 'COMUNA'})
df_vacunacion=df_vacunacion[['COMUNA','RangoEtareo','Primera Dosis','Segunda Dosis','Unica Dosis']].groupby(by=["COMUNA",'RangoEtareo'], dropna=False).sum().reset_index()

pob_censo_edad=pob_censo_edad.replace('*',0)

pob_censo_edad.dtypes

pob_censo_edad = pob_censo_edad.astype({
    'EDAD_0A5': 'int64',
    'EDAD_6A14': 'int64',
    'EDAD_15A64': 'int64',
    'EDAD_65YMAS':'int64'    
})
#
pob_censo_edad_gb=pob_censo_edad.groupby(by=["COMUNA"], dropna=True).sum().reset_index()
pob_censo_edadmelt=pd.melt(pob_censo_edad_gb, id_vars =['COMUNA'], value_vars =eiqueta,value_name='habitantes',var_name='RangoEtareo')
pob_censo_edadmelt["COMUNA"] = pd.to_numeric(pob_censo_edadmelt["COMUNA"])
hdi=hdi.rename(columns={'comuna': 'COMUNA'})
hdi=hdi[['COMUNA','HDI']]
hdi["COMUNA"] = pd.to_numeric(hdi["COMUNA"])

df = pd.merge(df_vacunacion,pob_censo_edadmelt,on=['COMUNA','RangoEtareo'],  how='left')
df = pd.merge(df,hdi,on=['COMUNA'],  how='left')
df=df.dropna()
df=df.set_index('COMUNA')

TypeError: group() got an unexpected keyword argument 'dropna'


### 3) Tratamiento de la correlación



In [16]:
df.loc[df['RangoEtareo']=='EDAD_0A5'].corr()




NameError: name 'df' is not defined

In [ ]:
df.loc[df['RangoEtareo']=='EDAD_6A14'].corr()

In [ ]:
df.loc[df['RangoEtareo']=='EDAD_15A64'].corr()

In [ ]:
df.loc[df['RangoEtareo']=='EDAD_15A64'].corr()

### 4) Gráficos